In [1]:
import numpy as np
from scipy.optimize import differential_evolution
import subprocess

In [2]:
def modify_and_run(params, param_template='template.param', param_file='paramSim.param'):
    # Unpack the parameters
    K, E, B, C = params
    
    # Read the template file
    with open(param_template, 'r') as file:
        content = file.readlines()
    
    # Replace placeholders with current parameters
    new_content = []
    for line in content:
        if '#K' in line:
            new_content.append(f"{2*int(K)}             # Average degree of nodes (even number)\n")
        elif '#E' in line:
            new_content.append(f"{E}             # Probability of developing infection\n")
        elif '#B' in line:
            new_content.append(f"{B}             # Probability of infecting\n")
        elif '#C' in line:
            new_content.append(f"{C}             # Prob. to restrict a random edge during the LD\n")
        elif '#SEED' in line:
            new_content.append(f"{np.random.randint(99999)}             # Seed (random numbers)\n")
        else:
            new_content.append(line)
    
    # Write modified content to the param file
    with open(param_file, 'w') as file:
        file.writelines(new_content)

    # Open the param file in binary mode and read content
    with open(param_file, 'rb') as file:
        param_data = file.read()
    
    # Run the simulation
    subprocess.run(["./EpiNetSimulator"], input=param_data, stdout=subprocess.PIPE)

In [3]:
def read_data(filename):
    data = np.loadtxt(filename)
    return data

In [4]:
nNodes = 1e5
data = read_data('peakUSA.data')
data = nNodes*data

def objective_function(params):
    modify_and_run(params)
    simulated = read_data('aveNewI.dat')
    rss = np.sum(np.square(data - simulated))
    return rss

In [ ]:
# Set bounds for K, E, B, C
bounds = [(1, 12),  # K
          (0.01, 1.0), # epsilon
          (0.01, 0.3), # beta
          (0.0, 1.0)]  # chi

result = differential_evolution(objective_function, bounds, strategy='best1bin', maxiter=100, popsize=50)

print('Optimal Parameters:', result.x)
print('Minimum RSS:', result.fun)

In [ ]:
|